In [1]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.lightgbm import LightGBM as Model

In [2]:
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
td.encode_last_positions()
td.encode()
td.drop()

In [5]:
att_features = ['sattact90','goals90','npgoals90','xg90','hdrgoals90','shots90','ontrgtpct','goalcnvrsnpct','assists90','crosses90',
                'acccrossespct','crossesleft90','acccrossesleftpct','crossesright90','acccrossesrightpct','crosses6yard90','dribbles90',
                'succssdribblespct','offduels90','offduelspct','touchesbox90','prgruns90','acclr90','rcvdpasses90','rcvdlongpasses90',
                'foulsa90','fwdpasses90','accfwdpassespct','xa90','shotassist90','secassist90','thirdassist90','smartpasses90',
                'accsmartpassespct','kp90','final3rdpasses90','accfinal3rdpassespct','penareapasses90','accpenareapassespct','thrpasses90',
                'accthrpassespct','directfk90','directfkontrgtpct','corners90','pens','pencnvrspct']
att_features2 = ['goals90','npgoals90','xg90','hdrgoals90','shots90','xa90','shotassist90','touchesbox90','prgruns90','acclr90','rcvdpasses90','smartpasses90', 'dribbles90',
                'succssdribblespct']

In [6]:
scores = pd.read_csv('catboost_att.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(3).to_dict()

In [7]:
model_scores = pd.DataFrame()
def pipeline(pos, features=None, full_feature_set=False):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    m = Model(data = td.data, features = features, full_feature_set=full_feature_set)
    display(m.data[m.features], features)
    m.train_test_split()
    m.scale()
    m.tune_hp()
    m.train()
    m.inverse_scale()
    global model_scores
    model_scores = pd.concat([model_scores, m.score(thresh=1.5)])
    # display(m.feature_importance())
    m.plot_predictions()
    m.top_n_predictions(20)
    display(m.top_n_predictions(20, worst=True))
    m.calculate_effectiveness()

In [8]:
pipeline('ATT')

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,deepcomp90,deepcompcrosses90,prpasses90,accprpassespct,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
0,28,0.0,1,120.0,1877,2000,1671.233333,1671.233333,130.0,37,...,2.46,0.43,5.16,76.68,1.20,0.4,40.00,1.63,3,100.0
1,19,0.0,1,127.2,1797,1877,1475.571429,1671.233333,70.0,26,...,1.44,0.69,4.32,79.31,0.05,0.0,0.00,0.05,0,0.0
7,26,0.0,1,74.0,1842,1733,1696.800000,1561.400000,75.0,32,...,1.43,0.54,2.17,85.71,0.00,0.0,0.00,0.00,0,0.0
10,21,0.0,1,63.0,1749,1840,1605.100000,1671.233333,60.0,32,...,1.07,0.27,3.82,77.00,0.00,0.0,0.00,0.00,1,100.0
13,25,0.0,1,50.0,1749,1733,1605.100000,1696.800000,40.0,29,...,1.28,0.19,2.76,77.46,0.00,0.0,0.00,0.00,3,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,0.73,0.94,4.19,72.50,0.00,0.0,0.00,0.21,0,0.0
2215,19,4.0,0,10.0,1788,1656,1653.333333,1552.633333,12.0,19,...,0.53,2.12,3.71,64.29,0.00,0.0,0.00,0.27,0,0.0
2219,31,4.0,0,1.7,1407,1316,1435.363636,1435.363636,2.4,27,...,0.63,0.21,2.86,48.53,0.00,0.0,0.00,0.04,2,50.0
2220,22,4.0,0,2.5,1524,1569,1474.736842,1552.633333,1.8,29,...,2.20,0.29,3.15,86.05,0.00,0.0,0.00,0.00,0,0.0


None

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 71, 'learning_rate': 0.07246463053980279, 'subsample': 0.7085412786200267, 'colsample_bytree': 0.6275286151967617, 'min_child_samples': 7, 'reg_alpha': 0.561958686908774, 'reg_lambda': 3.4291368226649976}
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018628 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13415
[LightGBM] [Info] Number of data points in the train set: 716, number of used features: 91
thresh = 1.5
mae: 5.1719601738443295
mape: 0.778709787695783
mae_below_thresh: 1.149228233075332
mape_above_thresh: 0.6307467690526072


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1270,Erling Haaland,21,22/23,Germany,L1,Borussia Dortmund,England,GB1,Manchester City,1,150.0,180.0,60.00,180.0,87.028101,92.971899
1560,Xavi Simons,19,22/23,France,FR1,Paris Saint-Germain,Netherlands,NL1,PSV Eindhoven,1,4.0,40.0,0.00,40.0,8.742160,31.257840
484,André Silva,24,20/21,Italy,IT1,AC Milan,Germany,L1,Eintracht Frankfurt,1,24.0,45.0,3.00,45.0,16.756044,28.243956
511,Trincão,20,20/21,Portugal,PO1,SC Braga,Spain,ES1,FC Barcelona,1,16.0,22.0,30.94,22.0,49.257152,27.257152
16,Rafael Leão,20,19/20,France,FR1,LOSC Lille,Italy,IT1,AC Milan,1,15.0,20.0,49.50,20.0,44.846736,24.846736
53,Pablo Fornals,23,19/20,Spain,ES1,Villarreal CF,England,GB1,West Ham United,1,25.0,16.0,28.00,16.0,38.129895,22.129895
51,Leonardo Spinazzola,26,19/20,Italy,IT1,Juventus FC,Italy,IT1,AS Roma,1,15.0,14.5,29.50,14.5,34.575619,20.075619
1722,Memphis Depay,28,22/23,Spain,ES1,FC Barcelona,Spain,ES1,Atlético de Madrid,0,35.0,14.0,3.00,14.0,33.263416,19.263416
495,Alexander Sørloth,24,20/21,England,GB1,Crystal Palace,Germany,L1,RB Leipzig,1,13.0,16.0,20.00,16.0,33.802221,17.802221
1851,Jesper Lindstrøm,23,23/24,Germany,L1,Eintracht Frankfurt,Italy,IT1,SSC Napoli,1,28.0,25.0,30.00,25.0,42.292952,17.292952


In [9]:
pipeline('ATT', att_features2, full_feature_set=False)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,shots90,xa90,shotassist90,touchesbox90,prgruns90,acclr90,rcvdpasses90,smartpasses90,dribbles90,succssdribblespct
0,28,0.0,1,120.0,1877,2000,1671.233333,1671.233333,130.0,37,...,2.86,0.18,1.55,3.37,0.99,0.78,28.79,2.17,2.06,44.16
1,19,0.0,1,127.2,1797,1877,1475.571429,1671.233333,70.0,26,...,3.18,0.20,1.34,5.56,0.84,0.89,26.16,1.79,3.18,53.13
7,26,0.0,1,74.0,1842,1733,1696.800000,1561.400000,75.0,32,...,2.17,0.06,0.70,4.03,0.70,0.19,11.66,0.58,3.25,48.81
10,21,0.0,1,63.0,1749,1840,1605.100000,1671.233333,60.0,32,...,3.21,0.13,0.88,4.21,0.99,0.73,14.07,1.19,3.52,51.09
13,25,0.0,1,50.0,1749,1733,1605.100000,1696.800000,40.0,29,...,1.87,0.20,1.05,3.42,0.51,0.27,14.86,1.01,2.72,44.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,0.52,0.16,1.26,1.88,3.98,1.99,27.10,0.10,6.38,47.54
2215,19,4.0,0,10.0,1788,1656,1653.333333,1552.633333,12.0,19,...,2.78,0.28,1.06,3.71,2.78,1.06,16.30,0.40,10.34,57.69
2219,31,4.0,0,1.7,1407,1316,1435.363636,1435.363636,2.4,27,...,1.68,0.06,0.55,3.03,0.67,0.46,17.70,0.21,1.93,43.48
2220,22,4.0,0,2.5,1524,1569,1474.736842,1552.633333,1.8,29,...,1.76,0.29,1.91,4.40,3.30,1.39,14.16,1.47,8.00,44.04


['goals90',
 'npgoals90',
 'xg90',
 'hdrgoals90',
 'shots90',
 'xa90',
 'shotassist90',
 'touchesbox90',
 'prgruns90',
 'acclr90',
 'rcvdpasses90',
 'smartpasses90',
 'dribbles90',
 'succssdribblespct']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 67, 'learning_rate': 0.0854060273223052, 'subsample': 0.9916413498624805, 'colsample_bytree': 0.8022608434752388, 'min_child_samples': 9, 'reg_alpha': 2.088406311658107, 'reg_lambda': 9.597783609907735}
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001555 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3180
[LightGBM] [Info] Number of data points in the train set: 716, number of used features: 27
thresh = 1.5
mae: 4.7211886019919
mape: 0.7696557820294091
mae_below_thresh: 1.2561854874170246
mape_above_thresh: 0.5840741181877536


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1270,Erling Haaland,21,22/23,Germany,L1,Borussia Dortmund,England,GB1,Manchester City,1,150.0,180.0,60.00,180.0,83.626725,96.373275
1560,Xavi Simons,19,22/23,France,FR1,Paris Saint-Germain,Netherlands,NL1,PSV Eindhoven,1,4.0,40.0,0.00,40.0,3.422238,36.577762
484,André Silva,24,20/21,Italy,IT1,AC Milan,Germany,L1,Eintracht Frankfurt,1,24.0,45.0,3.00,45.0,16.871739,28.128261
1851,Jesper Lindstrøm,23,23/24,Germany,L1,Eintracht Frankfurt,Italy,IT1,SSC Napoli,1,28.0,25.0,30.00,25.0,49.312847,24.312847
53,Pablo Fornals,23,19/20,Spain,ES1,Villarreal CF,England,GB1,West Ham United,1,25.0,16.0,28.00,16.0,39.032204,23.032204
495,Alexander Sørloth,24,20/21,England,GB1,Crystal Palace,Germany,L1,RB Leipzig,1,13.0,16.0,20.00,16.0,35.281010,19.281010
468,Rodrigo,29,20/21,Spain,ES1,Valencia CF,England,GB1,Leeds United,1,40.0,18.0,30.00,18.0,35.388631,17.388631
16,Rafael Leão,20,19/20,France,FR1,LOSC Lille,Italy,IT1,AC Milan,1,15.0,20.0,49.50,20.0,37.133806,17.133806
918,Luca Waldschmidt,25,21/22,Portugal,PO1,SL Benfica,Germany,L1,VfL Wolfsburg,1,15.0,7.0,12.00,7.0,23.897098,16.897098
1456,Takefusa Kubo,21,22/23,Spain,ES1,Real Madrid,Spain,ES1,Real Sociedad,1,7.5,25.0,6.50,25.0,9.667277,15.332723


In [10]:
scores = pd.read_csv('lightgbm_att.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(5).to_dict()

In [11]:
for _, features in feature_sets.items():
    pipeline(pos='ATT', features=features, full_feature_set=True)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,final3rdpasses90,longpasses90,accshortmedpassespct,thirdassist90,dribbles90,fk90,avglongpasslen,avgpasslen,assists90,shotblocks90
0,28,0.0,1,120.0,1877,2000,1671.233333,1671.233333,130.0,37,...,3.93,2.27,83.54,0.11,2.06,1.20,35.03,16.25,0.19,0.03
1,19,0.0,1,127.2,1797,1877,1475.571429,1671.233333,70.0,26,...,3.77,0.99,78.73,0.05,3.18,0.05,18.22,16.36,0.30,0.05
7,26,0.0,1,74.0,1842,1733,1696.800000,1561.400000,75.0,32,...,1.74,1.01,76.59,0.04,3.25,0.00,23.02,17.01,0.00,0.08
10,21,0.0,1,63.0,1749,1840,1605.100000,1671.233333,60.0,32,...,3.29,1.87,75.61,0.00,3.52,0.00,30.80,17.60,0.15,0.04
13,25,0.0,1,50.0,1749,1733,1605.100000,1696.800000,40.0,29,...,3.50,1.32,73.44,0.16,2.72,0.00,26.72,14.56,0.31,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,1.67,0.63,89.97,0.00,6.38,0.00,7.15,14.22,0.21,0.00
2215,19,4.0,0,10.0,1788,1656,1653.333333,1552.633333,12.0,19,...,0.93,0.93,76.19,0.00,10.34,0.00,8.48,17.03,0.27,0.00
2219,31,4.0,0,1.7,1407,1316,1435.363636,1435.363636,2.4,27,...,1.81,0.67,79.89,0.04,1.93,0.00,19.85,15.65,0.13,0.29
2220,22,4.0,0,2.5,1524,1569,1474.736842,1552.633333,1.8,29,...,2.35,0.22,73.17,0.00,8.00,0.00,4.11,11.96,0.37,0.07


['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'offduels90',
 'shots90',
 'acclr90',
 'accthrpassespct',
 'directfkontrgtpct',
 'aerduels90',
 'acclatpassespct',
 'directfk90',
 'secassist90',
 'acccrossesrightpct',
 'pencnvrspct',
 'shotassist90',
 'crossesleft90',
 'rcards90',
 'hdrgoals90',
 'pens',
 'final3rdpasses90',
 'longpasses90',
 'accshortmedpassespct',
 'thirdassist90',
 'dribbles90',
 'fk90',
 'avglongpasslen',
 'avgpasslen',
 'assists90',
 'shotblocks90']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 27, 'learning_rate': 0.0882762305938538, 'subsample': 0.6619275381760498, 'colsample_bytree': 0.7781496008185622, 'min_child_samples': 8, 'reg_alpha': 2.384272644131624, 'reg_lambda': 8.26137537948825}
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003450 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4276
[LightGBM] [Info] Number of data points in the train set: 716, number of used features: 39
thresh = 1.5
mae: 5.009287514198371
mape: 0.771225812749335
mae_below_thresh: 1.185195624990188
mape_above_thresh: 0.6029055845532069


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1270,Erling Haaland,21,22/23,Germany,L1,Borussia Dortmund,England,GB1,Manchester City,1,150.0,180.0,60.00,180.0,84.491297,95.508703
1560,Xavi Simons,19,22/23,France,FR1,Paris Saint-Germain,Netherlands,NL1,PSV Eindhoven,1,4.0,40.0,0.00,40.0,2.389755,37.610245
484,André Silva,24,20/21,Italy,IT1,AC Milan,Germany,L1,Eintracht Frankfurt,1,24.0,45.0,3.00,45.0,15.386445,29.613555
16,Rafael Leão,20,19/20,France,FR1,LOSC Lille,Italy,IT1,AC Milan,1,15.0,20.0,49.50,20.0,41.949966,21.949966
53,Pablo Fornals,23,19/20,Spain,ES1,Villarreal CF,England,GB1,West Ham United,1,25.0,16.0,28.00,16.0,36.160972,20.160972
1851,Jesper Lindstrøm,23,23/24,Germany,L1,Eintracht Frankfurt,Italy,IT1,SSC Napoli,1,28.0,25.0,30.00,25.0,44.708197,19.708197
511,Trincão,20,20/21,Portugal,PO1,SC Braga,Spain,ES1,FC Barcelona,1,16.0,22.0,30.94,22.0,40.969760,18.969760
468,Rodrigo,29,20/21,Spain,ES1,Valencia CF,England,GB1,Leeds United,1,40.0,18.0,30.00,18.0,36.951350,18.951350
1813,Matheus Nunes,25,23/24,England,GB1,Wolverhampton Wanderers,England,GB1,Manchester City,1,45.0,55.0,62.00,55.0,72.186451,17.186451
1713,Anthony Gordon,21,22/23,England,GB1,Everton FC,England,GB1,Newcastle United,0,20.0,38.0,45.60,38.0,55.036496,17.036496


...

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,directfkontrgtpct,aerduels90,acclatpassespct,directfk90,secassist90,acccrossesrightpct,pencnvrspct,shotassist90,crossesleft90,rcards90
0,28,0.0,1,120.0,1877,2000,1671.233333,1671.233333,130.0,37,...,40.00,3.02,80.00,0.4,0.00,20.69,100.0,1.55,0.56,0.00
1,19,0.0,1,127.2,1797,1877,1475.571429,1671.233333,70.0,26,...,0.00,3.52,71.58,0.0,0.15,37.50,0.0,1.34,0.74,0.00
7,26,0.0,1,74.0,1842,1733,1696.800000,1561.400000,75.0,32,...,0.00,7.01,65.13,0.0,0.19,29.41,0.0,0.70,0.70,0.00
10,21,0.0,1,63.0,1749,1840,1605.100000,1671.233333,60.0,32,...,0.00,5.66,74.85,0.0,0.15,25.00,100.0,0.88,0.61,0.00
13,25,0.0,1,50.0,1749,1733,1605.100000,1696.800000,40.0,29,...,0.00,14.08,74.36,0.0,0.04,42.86,100.0,1.05,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214,18,4.0,0,17.0,1612,1696,1474.736842,1552.633333,2.8,22,...,0.00,0.94,89.22,0.0,0.10,27.27,0.0,1.26,1.88,0.00
2215,19,4.0,0,10.0,1788,1656,1653.333333,1552.633333,12.0,19,...,0.00,3.05,61.54,0.0,0.00,58.33,0.0,1.06,2.92,0.00
2219,31,4.0,0,1.7,1407,1316,1435.363636,1435.363636,2.4,27,...,0.00,5.93,78.98,0.0,0.08,33.33,50.0,0.55,0.29,0.00
2220,22,4.0,0,2.5,1524,1569,1474.736842,1552.633333,1.8,29,...,0.00,3.89,76.06,0.0,0.07,22.22,0.0,1.91,0.51,0.00


['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'offduels90',
 'shots90',
 'acclr90',
 'accthrpassespct',
 'directfkontrgtpct',
 'aerduels90',
 'acclatpassespct',
 'directfk90',
 'secassist90',
 'acccrossesrightpct',
 'pencnvrspct',
 'shotassist90',
 'crossesleft90',
 'rcards90']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 21, 'learning_rate': 0.08669010952647005, 'subsample': 0.5818857666137643, 'colsample_bytree': 0.735583820342118, 'min_child_samples': 9, 'reg_alpha': 1.5977739438978094, 'reg_lambda': 9.944565798809283}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001248 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2700
[LightGBM] [Info] Number of data points in the train set: 716, number of used features: 27
thresh = 1.5
mae: 4.716301367581378
mape: 0.7584263008701349
mae_below_thresh: 1.2160880676634396
mape_above_thresh: 0.5765150095315879


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1270,Erling Haaland,21,22/23,Germany,L1,Borussia Dortmund,England,GB1,Manchester City,1,150.0,180.0,60.0,180.0,87.127373,92.872627
1560,Xavi Simons,19,22/23,France,FR1,Paris Saint-Germain,Netherlands,NL1,PSV Eindhoven,1,4.0,40.0,0.0,40.0,2.456370,37.543630
484,André Silva,24,20/21,Italy,IT1,AC Milan,Germany,L1,Eintracht Frankfurt,1,24.0,45.0,3.0,45.0,15.725335,29.274665
53,Pablo Fornals,23,19/20,Spain,ES1,Villarreal CF,England,GB1,West Ham United,1,25.0,16.0,28.0,16.0,40.347020,24.347020
1851,Jesper Lindstrøm,23,23/24,Germany,L1,Eintracht Frankfurt,Italy,IT1,SSC Napoli,1,28.0,25.0,30.0,25.0,47.442476,22.442476
468,Rodrigo,29,20/21,Spain,ES1,Valencia CF,England,GB1,Leeds United,1,40.0,18.0,30.0,18.0,38.996307,20.996307
495,Alexander Sørloth,24,20/21,England,GB1,Crystal Palace,Germany,L1,RB Leipzig,1,13.0,16.0,20.0,16.0,34.762852,18.762852
51,Leonardo Spinazzola,26,19/20,Italy,IT1,Juventus FC,Italy,IT1,AS Roma,1,15.0,14.5,29.5,14.5,30.410544,15.910544
16,Rafael Leão,20,19/20,France,FR1,LOSC Lille,Italy,IT1,AC Milan,1,15.0,20.0,49.5,20.0,35.846711,15.846711
1722,Memphis Depay,28,22/23,Spain,ES1,FC Barcelona,Spain,ES1,Atlético de Madrid,0,35.0,14.0,3.0,14.0,28.088875,14.088875


In [12]:
with pd.option_context('display.max_colwidth', None):
    display(model_scores.sort_values('mape_above_thresh'))

,mae,mape,mae_below_thresh,mape_above_thresh,features
0,4.649500,0.742342,1.288949,0.531224,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduels90, shots90, acclr90, accthrpassespct, directfkontrgtpct, aerduels90, acclatpassespct, directfk90, secassist90, acccrossesrightpct, pencnvrspct, shotassist90, crossesleft90, rcards90, hdrgoals90]"
0,4.669117,0.818794,1.522514,0.573367,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduels90, shots90, acclr90, accthrpassespct, directfkontrgtpct, aerduels90, acclatpassespct, directfk90, secassist90, acccrossesrightpct, pencnvrspct, shotassist90, crossesleft90, rcards90, hdrgoals90, pens]"
0,4.716301,0.758426,1.216088,0.576515,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduels90, shots90, acclr90, accthrpassespct, directfkontrgtpct, aerduels90, acclatpassespct, directfk90, secassist90, acccrossesrightpct, pencnvrspct, shotassist90, crossesleft90, rcards90]"
0,4.721189,0.769656,1.256185,0.584074,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, goals90, npgoals90, xg90, hdrgoals90, shots90, xa90, shotassist90, touchesbox90, prgruns90, acclr90, rcvdpasses90, smartpasses90, dribbles90, succssdribblespct]"
0,4.805690,0.707087,0.941648,0.598178,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduels90, shots90, acclr90, accthrpassespct, directfkontrgtpct, aerduels90, acclatpassespct, directfk90, secassist90, acccrossesrightpct, pencnvrspct, shotassist90, crossesleft90, rcards90, hdrgoals90, pens, final3rdpasses90, longpasses90, accshortmedpassespct, thirdassist90, dribbles90, fk90, avglongpasslen, avgpasslen, assists90, shotblocks90, accfwdpassespct]"
0,5.009288,0.771226,1.185196,0.602906,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, offduels90, shots90, acclr90, accthrpassespct, directfkontrgtpct, aerduels90, acclatpassespct, directfk90, secassist90, acccrossesrightpct, pencnvrspct, shotassist90, crossesleft90, rcards90, hdrgoals90, pens, final3rdpasses90, longpasses90, accshortmedpassespct, thirdassist90, dribbles90, fk90, avglongpasslen, avgpasslen, assists90, shotblocks90]"
0,5.171960,0.778710,1.149228,0.630747,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, duels90, duelswonpct, foot, height, weight, succssdefact90, defduels90, defduelswpct, aerduels90, aerduelspct, sltackles90, padjsltackles, shotblocks90, intrcpt90, padjintrcpt, fouls90, ycards90, rcards90, sattact90, goals90, npgoals90, xg90, hdrgoals90, shots90, ontrgtpct, goalcnvrsnpct, assists90, crosses90, acccrossespct, crossesleft90, acccrossesleftpct, crossesright90, acccrossesrightpct, crosses6yard90, dribbles90, succssdribblespct, offduels90, offduelspct, touchesbox90, prgruns90, acclr90, rcvdpasses90, rcvdlongpasses90, foulsa90, passes90, accpassespct, fwdpasses90, accfwdpassespct, bpasses90, accbpassespct, latpasses90, acclatpassespct, shortmedpasses90, accshortmedpassespct, longpasses90, acclongpassesct, avgpasslen, avglongpasslen, xa90, shotassist90, secassist90, thirdassist90, smartpasses90, accsmartpassespct, kp90, final3rdpasses90, accfinal3rdpassespct, penareapasses90, accpenareapassespct, thrpasses90, accthrpassespct, deepcomp90, deepcompcrosses90, prpasses90, accprpassespct, fk90, directfk90, directfkontrgtpct, corners90, pens, pencnvrspct]"
